In [76]:
import numpy as np
from numpy.typing import NDArray
from ska_sdp_datamodels.visibility import Visibility as RASCILVisibility
from typing_extensions import assert_never
import os
from karabo.imaging.imager_base import DirtyImagerConfig
from karabo.imaging.imager_oskar import OskarDirtyImager
from karabo.imaging.imager_rascil import RascilDirtyImager,RascilDirtyImagerConfig
from karabo.imaging.util import (
    auto_choose_dirty_imager_from_sim,
    auto_choose_dirty_imager_from_vis,
)
from karabo.imaging.imager_wsclean import ( # type: ignore
    WscleanDirtyImager,
    WscleanImageCleaner,
    WscleanImageCleanerConfig,
    create_image_custom_command,
)
from karabo.imaging import imager_wsclean
from karabo.simulation.interferometer import InterferometerSimulation
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope
from karabo.simulation.visibility import Visibility
from karabo.imaging.imager_oskar import OskarDirtyImager, OskarDirtyImagerConfig
from karabo.simulator_backend import SimulatorBackend
from astropy.io import fits
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib as mpl
from karabo.imaging.image import ImageMosaicker
from oskar.measurement_set import MeasurementSet

mpl.use('Tkagg')

from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
import ARatmospy.ArScreens as ArScreens


In [198]:
############# Code Cell for SKA-mid and Meerkat Configuration ####################
#telescope_path = '/media/rohit/sdata/ska-solar-files/meerkat.tm'
#telescope_path = '/media/rohit/sdata/ska-solar-files/ska1mid.tm'
telescope_path = '/media/rohit/sdata/ska-solar-files/mwa.phase1.tm'
file_name='mightee_meerkat_snapshot2'
noise_enable_ = False
enable_array_beam=False
#path = '/media/rohit/sdata/karabo-paper-files/sim/5h_meerkat'
#path = '/media/rohit/sdata/karabo-paper-files/sim/5h_skamid'
#path = '/media/rohit/sdata/karabo-paper-files/sim/snapshot_meerkat'
path = '/media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid'
print('#### TELESCOPE PATH: ',telescope_path)
print('#### File Name: ',file_name)
sky = SkyModel()
mightee1 = SkyModel.get_MIGHTEE_Sky()
mightee_continuum = mightee1.to_np_array()
sky_data = np.zeros((len(mightee_continuum), 12))
sky_data[:, 0] = mightee_continuum[:, 0]
sky_data[:, 1] = mightee_continuum[:, 1]
sky_data[:, 2] = mightee_continuum[:, 2]
sky_data[:, 6] = mightee_continuum[:, 6]
sky_data[:, 9] = mightee_continuum[:, 9]
sky_data[:, 10] = mightee_continuum[:, 10]
sky_data[:, 11] = mightee_continuum[:, 11]
sky.add_point_sources(sky_data)
ra_list = [150.0];dec_list = [2.0];f_obs = 1.56e8;chan = 0.2e6 # at SKA-mid frequencies
sec_ = 8;t_int = 8; nchan =2 # for SKA mid freq
hours_= 0
min_= 0 # 1 hour pointing
t_nchan = int((hours_*3600.0+min_*60.+sec_)/t_int)

print(' Number of sky sources:',sky_data.shape[0],' Number of time channels:',t_nchan,' Number of freqeuncy channels:',nchan)


#### TELESCOPE PATH:  /media/rohit/sdata/ska-solar-files/mwa.phase1.tm
#### File Name:  mightee_meerkat_snapshot2
 Number of sky sources: 9896  Number of time channels: 1  Number of freqeuncy channels: 2


In [ ]:
ion=1
if(ion==1):
    screen_width_metres = 200e3
    r0 = 5.0e3  # Scale size (5 km).
    bmax = 20e3  # 20 km sub-aperture size.
    sampling = 100.0  # 100 m/pixel.
    m = int(bmax / sampling)  # Pixels per sub-aperture (200).
    n = int(screen_width_metres / bmax)  # Sub-apertures across the screen (10).
    num_pix = n * m
    pscale = screen_width_metres / (n * m)  # Pixel scale (100 m/pixel).
    print("Number of pixels %d, pixel size %.3f m" % (num_pix, pscale))
    print("Field of view %.1f (m)" % (num_pix * pscale))
    speed = 150e3 / 3600.0  # 150 km/h in m/s.
    # Parameters for each layer.
    # (scale size [m], speed [m/s], direction [deg], layer height [m]).
    layer_params = np.array([(r0, speed, 60.0, 300e3),
                                (r0, speed/2.0, -30.0, 310e3)])

    rate = 1.0/60.0  # The inverse frame rate (1 per minute).
    alpha_mag = 0.999  # Evolve screen slowly.
    num_times = 5  # Four hours.
    my_screens = ArScreens.ArScreens(n, m, pscale, rate, layer_params, alpha_mag)
    my_screens.run(num_times)

    # Convert to TEC
    # phase = image[pixel] * -8.44797245e9 / frequency
    frequency = f_obs
    phase2tec = -frequency / 8.44797245e9 

    w = WCS(naxis=4)
    w.naxis = 4
    w.wcs.cdelt = [pscale, pscale, 1.0 / rate, 1.0]
    w.wcs.crpix = [num_pix // 2 + 1, num_pix // 2 + 1, num_times // 2 + 1, 1.0]
    w.wcs.ctype = ['XX', 'YY', 'TIME', 'FREQ']
    w.wcs.crval = [0.0, 0.0, 0.0, frequency]
    data = np.zeros([1, num_times, num_pix, num_pix])
    print(data.shape)
    for layer in range(len(my_screens.screens)):
        for i, screen in enumerate(my_screens.screens[layer]):
            data[:, i, ...] += phase2tec * screen[np.newaxis, ...]
    print(data.shape)
    fits.writeto(filename=path+'/atmo/test_screen_60s.fits', data=data,
                header=w.to_header(), overwrite=True)
aa,ha=fits.getdata(path+'/atmo/test_screen_60s.fits',header=True)

Number of pixels 2000, pixel size 100.000 m
Field of view 200000.0 (m)
(1, 5, 2000, 2000)
(1, 5, 2000, 2000)
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                  -64 / array data type                                NAXIS   =                    4 / number of array dimensions                     NAXIS1  =                 2000                                                  NAXIS2  =                 2000                                                  NAXIS3  =                    5                                                  NAXIS4  =                    1                                                  WCSAXES =                    4 / Number of coordinate axes                      CRPIX1  =               1001.0 / Pixel coordinate of reference point            CRPIX2  =               1001.0 / Pixel coordinate of reference point            CRPIX3  =                  3.0 / Pixel coordinate of reference point            CRPIX4  =  

In [201]:

print(' Number of sky sources:',sky_data.shape[0],' Number of time channels:',t_nchan,' Number of freqeuncy channels:',nchan)
#---------------------------------
layout=np.loadtxt(telescope_path+'/layout.txt')
nant=len(layout)
nb=int(nant*(nant-1)*0.5)
print('Number of Baselines:',nb)
base_length=[0]*nb
k=0
for i in range(nant):
    for j in range(i,nant):
        if(i!=j):
            base_length[k] = np.sqrt((layout[i][0]-layout[j][0])**2 + (layout[i][1]-layout[j][1])**2)
            k=k+1
base_length=np.array(base_length)
beam_size_arcsec = 3.e8/f_obs/base_length.max()*180/np.pi*3600
print('Maximum Baseline (m)):',base_length.max(),' Beam Size (arcsec):',beam_size_arcsec)
img_nsize = 4096
cellsize_arcsec=3 #beam_size_arcsec/3 # in arcsec
cellsize_rad = cellsize_arcsec/3600.*np.pi/180.
print('BEAM SIZE: ',beam_size_arcsec,' arcsec',' | IMAGE CELL SIZE: ',cellsize_arcsec,' arcsec','| IMG SIZE:',img_nsize,' pixels','IMG FOV',img_nsize*cellsize_arcsec/3600.,' deg')
print('##---------- RA-DEC Iteration begins -----------##')
k=0
for phase_ra in ra_list:
    for phase_dec in dec_list:
        print('Iteration:',k,'| R.A.:',phase_ra,'deg | DEC:',phase_dec,'deg')
        sky_filter = sky.filter_by_radius(
            ra0_deg=phase_ra,
            dec0_deg=phase_dec,
            inner_radius_deg=0,
            outer_radius_deg=100.0,
        )
        # Skip analysis if there are no sources in this iteration
        if sky_filter.sources is None:
            continue
        r0 = 5e3  # Scale size (5 km).
        bmax = 20e3  # 20 km sub-aperture size.
        sampling = 100.0  # 100 m/pixel.
        telescope=Telescope.read_OSKAR_tm_file(telescope_path)
        telescope.read_OSKAR_tm_file(telescope_path)
        simulation = InterferometerSimulation(
            channel_bandwidth_hz=chan,
            time_average_sec=1,
            noise_enable=False,
            noise_seed="time",
            noise_freq="Range",
            noise_rms="Range",
            noise_start_freq=f_obs,
            noise_inc_freq=chan,
            noise_number_freq=1,
            noise_rms_start=5,
            noise_rms_end=10,
            ms_file_path=path+'/'+file_name+str(k)+'.ms',
            vis_path=path+'/'+file_name+str(k)+'.vis',
            ionosphere_screen_type = 'External',
            ionosphere_fits_path=path+'/atmo/test_screen_60s.fits',
            ionosphere_screen_height_km=r0,
            ionosphere_screen_pixel_size_m=sampling,
            ionosphere_isoplanatic_screen=True,
        )
        observation = Observation(
            phase_centre_ra_deg=phase_ra,
            start_date_and_time=datetime(2022, 9, 1, 9, 00, 00, 521489),
            length=timedelta(hours=hours_, minutes=min_, seconds=sec_, milliseconds=0),
            phase_centre_dec_deg=phase_dec,
            number_of_time_steps=t_nchan,
            start_frequency_hz=f_obs,
            frequency_increment_hz=chan,
            number_of_channels=nchan,
        )
        print('--- Simulation Run Begins....')
        visibility = simulation.run_simulation(
            telescope, sky, observation
        )
        print(visibility)
        print('--- Simulation Run Ends....')
        k=k+1
        
"""         dirty_imager = RascilDirtyImager(
            RascilDirtyImagerConfig(
                imaging_npixel=img_nsize,
                imaging_cellsize=cellsize_rad,
                combine_across_frequencies=False,
            )
        )
        
        dirty = dirty_imager.create_dirty_image(visibility)

        dirty.write_to_file(
            path+'mightee_'+file_name+str(k)+'.fits',
            overwrite=True,
        ) """


 Number of sky sources: 9896  Number of time channels: 1  Number of freqeuncy channels: 2
Number of Baselines: 8128
Maximum Baseline (m)): 2873.5020030311443  Beam Size (arcsec): 138.04169564465224
BEAM SIZE:  138.04169564465224  arcsec  | IMAGE CELL SIZE:  3  arcsec | IMG SIZE: 4096  pixels IMG FOV 3.4133333333333336  deg
##---------- RA-DEC Iteration begins -----------##
Iteration: 0 | R.A.: 150.0 deg | DEC: 2.0 deg
Parameter 'use_gpus' is None! Using function 'karabo.util.gpu_util.is_cuda_available()'. To overwrite, set 'use_gpus' True or False.
--- Simulation Run Begins....
Saved visibility to /media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid/mightee_meerkat_snapshot20.vis
--- Simulation Run Ends....


"         dirty_imager = RascilDirtyImager(\n            RascilDirtyImagerConfig(\n                imaging_npixel=img_nsize,\n                imaging_cellsize=cellsize_rad,\n                combine_across_frequencies=False,\n            )\n        )\n        \n        dirty = dirty_imager.create_dirty_image(visibility)\n\n        dirty.write_to_file(\n            path+'mightee_'+file_name+str(k)+'.fits',\n            overwrite=True,\n        ) "

In [202]:
res_read1=MeasurementSet.open(visibility.ms_file_path,readonly=True)
num_baselines_=int(res_read1.num_stations*(res_read1.num_stations-1)/2.)
uu,vv,ww=res_read1.read_coords(start_row=0,num_baselines=num_baselines_)
vis1=res_read1.read_vis(start_row=0,start_channel=0,num_channels=1,num_baselines=num_baselines_)
print(vis1.shape,vis1.mean())

res_read2=MeasurementSet.open('/media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid/mightee_meerkat_snapshot10.ms',readonly=True)
num_baselines_=int(res_read2.num_stations*(res_read2.num_stations-1)/2.)
uu,vv,ww=res_read2.read_coords(start_row=0,num_baselines=num_baselines_)
vis2=res_read2.read_vis(start_row=0,start_channel=0,num_channels=1,num_baselines=num_baselines_)
print(vis2.shape,vis2.mean())

(1, 8128, 4) (0.09897516+0.004427728j)
(1, 8128, 4) (0.09897516+0.004427727j)


In [179]:
nchan_img = nchan
img_nsize = 4096
cellsize_arcsec=15  #beam_size_arcsec/3 # in arcsec
cellsize_rad = cellsize_arcsec/3600.*np.pi/180.
print('BEAM SIZE: ',beam_size_arcsec,' arcsec',' | IMAGE CELL SIZE: ',cellsize_arcsec,' arcsec','| IMG SIZE:',img_nsize,' pixels','IMG FOV',img_nsize*cellsize_arcsec/3600.,' deg')

k=0
print('----OUTPUT MS PATH:',visibility.ms_file_path,'-----------')
output_img = path+'/img/'+file_name+str(k+1)+'.fits'
print('----OUTPUT IMG PATH:',output_img,'-----------')
print('##### IMAGING Begins.....')
img_cmd = 'wsclean \
         -size '+str(img_nsize)+' '+str(img_nsize)+' \
        -name '+output_img+' \
         -scale '+str(cellsize_rad)+'rad -niter 150 -mgain 0.8 \
        -channels-out '+str(nchan_img)+' '+visibility.ms_file_path
print(img_cmd)
os.system('export OPENBLAS_NUM_THREADS=1')
restored = imager_wsclean.create_image_custom_command(command=img_cmd)

BEAM SIZE:  138.04169564465224  arcsec  | IMAGE CELL SIZE:  15  arcsec | IMG SIZE: 4096  pixels IMG FOV 17.066666666666666  deg
----OUTPUT MS PATH: /media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid/mightee_meerkat_snapshot10.ms -----------
----OUTPUT IMG PATH: /media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid/img/mightee_meerkat_snapshot11.fits -----------
##### IMAGING Begins.....
wsclean          -size 4096 4096         -name /media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid/img/mightee_meerkat_snapshot11.fits          -scale 7.27220521664304e-05rad -niter 150 -mgain 0.8         -channels-out 2 /media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid/mightee_meerkat_snapshot10.ms
Creating /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-fdaEnl2y2R for Disk cache for WSClean custom command images
WSClean command: [cd /tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-fdaEnl2y2R && OPENBLAS_NUM_THREADS=1 wsclean          -size 4096 4

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/karabo-STM-rohit-TRnaSBphUX/WSClean-custom-fdaEnl2y2R/wsclean-image.fits'

In [ ]:
d1,h=fits.getdata('/media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid/img/mightee_meerkat_snapshot11.fits-0001-image.fits',header=True)
d2,h=fits.getdata('/media/rohit/sdata/karabo-paper-files/sim/wsclean_snapshot_skamid/img/mightee_meerkat_snapshot21.fits-0001-image.fits',header=True)
#d,h=fits.getdata(path+'wsclean_img.fits',header=True)
d1=d1[0,0]
d2=d2[0,0]
nsize = d1.shape[0] 
delt=h['CDELT1']
l1 = h['CRVAL1'] + h['CDELT1']*nsize
l2 = h['CRVAL1'] - h['CDELT1']*nsize
r1 = h['CRVAL2'] + h['CDELT2']*nsize 
r2 = h['CRVAL2'] - h['CDELT2']*nsize
npix_per_beam = beam_size_arcsec/cellsize_arcsec
wcs = WCS(h)
img_rms1 = d1[0:300,0:300].std()
img_max1 = d1.max()
img_dr1 = img_max1/img_rms1
img_rms2 = d2[0:300,0:300].std()
img_max2 = d2.max()
img_dr2 = img_max2/img_rms2
print('Without Ionosphere: RMS, MAX, DR: ',img_rms1, img_max1, img_dr1)
print('With Ionosphere: RMS, MAX, DR: ',img_rms2, img_max2, img_dr2)

Without Ionosphere: RMS, MAX, DR:  0.0014488233 0.048694782 33.609886
With Ionosphere: RMS, MAX, DR:  0.0014488233 0.048694786 33.609886


In [ ]:
mpl.use('Tkagg')
#d1,h=fits.getdata('/media/rohit/sdata/karabo-paper-files/sim/5h_meerkat/img/mightee_meerkat_long01.fits-image.fits',header=True)
d1,h=fits.getdata('/media/rohit/sdata/karabo-paper-files/sim/snapshot_skamid/img/mightee_meerkat_snapshot00.fits-0000-image.fits',header=True)
d=d1[0,0]
nsize = d[0].shape[0]
delt=h['CDELT1']
wcs = WCS(h)
print(wcs)
print(nsize,delt, wcs.pixel_n_dim)
ra_grid,dec_grid=np.meshgrid((np.arange(nsize)-int(nsize/2))*delt,(np.arange(nsize)-int(nsize/2))*delt)
ra_grid = ra_grid + h['CRVAL1']
dec_grid = dec_grid + h['CRVAL2']

f,ax0=plt.subplots(1,1,subplot_kw={'projection': wcs,'slices':('x','y',0,0)})
im0=ax0.imshow(d,origin='lower',
               cmap='YlGnBu',vmin=0.000005, vmax=0.002)#,norm=mpl.colors.LogNorm(vmin=0.0005))
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='5%', pad=0.0)
caxx=cax.coords[0]
caxy=cax.coords[1]
caxx.grid(False)
caxy.grid(False)
cax.tick_params(direction='out')
caxx.set_ticks(alpha=0, color='w', size=0, values=[0]*u.dimensionless_unscaled)
caxy.set_ticklabel_position('r')
caxx.set_auto_axislabel(False)
caxy.set_auto_axislabel(False)
cax.set_title('Flux Density (Jy)')
f.colorbar(im0, cax=cax, orientation='vertical')
ax0.set_xlabel('R.A. (hms)');ax0.set_ylabel('DEC (deg)')
plt.show()


In [ ]:
# Creating WCS for plotting Meerkat / SKA-mid

wcs2 = WCS(naxis=2)
wcs2.wcs.crval = wcs.wcs.crval[0:2] # Central coordinates (RA, Dec) in degrees
wcs2.wcs.cdelt = wcs.wcs.cdelt[0:2]  # Pixel scale (deg/pix)

'''wcs2.wcs.cdelt[0]=0.1
wcs2.wcs.cdelt[1]=0.1
wcs2.wcs.crval[0]=0
wcs2.wcs.crval[1]=0
wcs2.wcs.crpix[0] = 0
wcs2.wcs.crpix[1] = 0'''
#wcs2.wcs.crpix = wcs.wcs.crpix[0:2]  # Reference pixel
wcs2.wcs.ctype[0] = wcs.wcs.ctype[0]
wcs2.wcs.ctype[1] = wcs.wcs.ctype[1]
wcs2.wcs.pc = np.array([[1.0, 0.0],  # No rotation between world coordinates and pixel coordinates
                       [0.0, 1.0]])
print('WCS2',wcs2)

sky_data_world_coords = np.array([sky_data[:,0],sky_data[:,1]]).T
print(sky_data_world_coords.shape)
# Convert world coordinates (RA, Dec, freq, pol) to pixel coordinates
# The second argument `0` indicates that the pixel coordinates are 0-indexed
sky_data_pixel_coords = wcs2.wcs_world2pix(sky_data_world_coords, 0)
print(sky_data_pixel_coords[:,0])


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable  # Corrected import
import numpy as np
from astropy.wcs import WCS
import astropy.units as u

# Create the figure and axis with WCS projection
f, ax = plt.subplots(1, 1, subplot_kw={'projection': wcs2, 'slices': ('x', 'y')})

# Scatter plot using RA, Dec for positions and flux for point size
cm = plt.colormaps['Blues']
im1 = ax.scatter(sky_data_pixel_coords[:, 0], sky_data_pixel_coords[:, 1], c=sky_data[:, 2], s=sky_data[:, 2] * 1000,
                 vmax=0.0001, cmap=cm, alpha=0.6)

# Add colorbar
cbar = plt.colorbar(im1, ax=ax, orientation='vertical')  # Create the colorbar
cbar.set_label('Flux Density (Jy)', fontsize=12)  # Set the colorbar label

# Set the axis labels and invert the x-axis
ax.set_xlabel('RA (hms)')
ax.set_ylabel('Dec (deg)')
#ax.invert_xaxis()

# Save the plot
plt.savefig('/media/rohit/data/Dropbox/projects/SKA/SKA_project_fhnw/manuscript/meerkat_input_sky.png', dpi=150)

# Show the plot
plt.show()



GLEAM Simulation Ahead 

In [ ]:
sky = SkyModel()
gleam1 = SkyModel.get_GLEAM_Sky()
gleam_continuum = gleam1.to_np_array()
sky_data = np.zeros((len(gleam_continuum), 12))
sky_data[:, 0] = gleam_continuum[:, 0]
sky_data[:, 1] = gleam_continuum[:, 1]
sky_data[:, 2] = gleam_continuum[:, 2]
sky_data[:, 6] = gleam_continuum[:, 6]
sky_data[:, 9] = gleam_continuum[:, 9]
sky_data[:, 10] = gleam_continuum[:, 10]
sky_data[:, 11] = gleam_continuum[:, 11]
sky.add_point_sources(sky_data)
ra_list = [292.5];dec_list = [-55.0];f_obs = 72.0e6;chan = 40.0e3*256
phase_ra = ra_list[0];phase_dec = dec_list[0]
sky_filter = sky.filter_by_radius(
    ra0_deg=phase_ra,
    dec0_deg=phase_dec,
    inner_radius_deg=0,
    outer_radius_deg=6.0,
)
data_gleam = sky_filter.to_np_array()
mpl.use('Tkagg')

wcs2 = WCS(naxis=2)
wcs2.wcs.crval = wcs.wcs.crval[0:2] # Central coordinates (RA, Dec) in degrees
wcs2.wcs.cdelt = wcs.wcs.cdelt[0:2]  # Pixel scale (deg/pix)

'''wcs2.wcs.cdelt[0]=0.1
wcs2.wcs.cdelt[1]=0.1
wcs2.wcs.crval[0]=0
wcs2.wcs.crval[1]=0
wcs2.wcs.crpix[0] = 0
wcs2.wcs.crpix[1] = 0'''
#wcs2.wcs.crpix = wcs.wcs.crpix[0:2]  # Reference pixel
wcs2.wcs.ctype[0] = wcs.wcs.ctype[0]
wcs2.wcs.ctype[1] = wcs.wcs.ctype[1]
wcs2.wcs.pc = np.array([[1.0, 0.0],  # No rotation between world coordinates and pixel coordinates
                       [0.0, 1.0]])
print('WCS2',wcs2)

sky_data_world_coords = np.array([data_gleam[:,0],data_gleam[:,1]]).T
print(sky_data_world_coords.shape)
# Convert world coordinates (RA, Dec, freq, pol) to pixel coordinates
# The second argument `0` indicates that the pixel coordinates are 0-indexed
sky_data_pixel_coords = wcs2.wcs_world2pix(sky_data_world_coords, 0)
print(sky_data_pixel_coords[:,0],data_gleam[:,2].max())


In [ ]:
mpl.use('Tkagg')
d1,h=fits.getdata('/media/rohit/sdata/karabo-paper-files/sim/snapshot_mwa/img/gleam_mwa_snapshot00.fits-MFS-image.fits',header=True)
d=d1[0,0]
nsize = d[0].shape[0]
delt=h['CDELT1']
wcs = WCS(h)
print(nsize,delt, wcs.pixel_n_dim)
ra_grid,dec_grid=np.meshgrid((np.arange(nsize)-int(nsize/2))*delt,(np.arange(nsize)-int(nsize/2))*delt)
ra_grid = ra_grid + h['CRVAL1']
dec_grid = dec_grid + h['CRVAL2']



f,ax1=plt.subplots(1,1,subplot_kw={'projection': wcs2,'slices':('x','y')})
cm = plt.cm.get_cmap('Blues')
im1=ax1.scatter(sky_data_pixel_coords[:, 0],sky_data_pixel_coords[:, 1],c=data_gleam[:,2], s=data_gleam[:,2]*10, vmax=10, cmap=cm,alpha=0.6)
divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.0)
caxx=cax.coords[0]
caxy=cax.coords[1]
caxx.grid(False)
caxy.grid(False)
cax.tick_params(direction='out')
caxx.set_ticks(alpha=0, color='w', size=0, values=[0]*u.dimensionless_unscaled)
caxy.set_ticklabel_position('r')
caxx.set_auto_axislabel(False)
caxy.set_auto_axislabel(False)
cax.set_title('Flux Density (Jy)')
f.colorbar(im1, cax=cax, orientation='vertical')
ax1.set_xlabel('R.A. (hms)');ax1.set_ylabel('DEC (deg)')
plt.savefig('/media/rohit/data/Dropbox/projects/SKA/SKA_project_fhnw/manuscript/input_gleam_sky.png', dpi=150)

plt.show()

In [ ]:


f,ax0=plt.subplots(1,1,subplot_kw={'projection': wcs,'slices':('x','y',0,0)})
im0=ax0.imshow(d,origin='lower',
               cmap='YlGnBu',vmin=0.001,vmax=20)#,norm=mpl.colors.LogNorm(vmin=0.5))
divider = make_axes_locatable(ax0)
cax = divider.append_axes('right', size='5%', pad=0.0)
caxx=cax.coords[0]
caxy=cax.coords[1]
caxx.grid(False)
caxy.grid(False)
cax.tick_params(direction='out')
caxx.set_ticks(alpha=0, color='w', size=0, values=[0]*u.dimensionless_unscaled)
caxy.set_ticklabel_position('r')
caxx.set_auto_axislabel(False)
caxy.set_auto_axislabel(False)
cax.set_title('Flux Density (Jy)')
f.colorbar(im0, cax=cax, orientation='vertical')
ax0.set_xlabel('R.A. (hms)');ax0.set_ylabel('DEC (deg)')
plt.show()
